In [1]:
var RankingMetrics = require('eclairjs/mllib/evaluation').RankingMetrics;
var RegressionMetrics = require('eclairjs/mllib/evaluation').RegressionMetrics;
var ALS = require('eclairjs/mllib/recommendation/ALS');
var Rating = require('eclairjs/mllib/recommendation/Rating');
var List = require('eclairjs/List');
var Tuple = require('eclairjs/Tuple');
var PairRDD = require('eclairjs/PairRDD');


In [2]:
var filename =  "../../data/mllib/sample_movielens_data.txt";

    var SparkConf = require('eclairjs/SparkConf');
    var SparkContext = require('eclairjs/SparkContext');
    var conf = new SparkConf().setAppName("Ranking Metrics Example");
    var sc = new SparkContext(conf);

    var data = data = sc.textFile(filename);

    var ratings = data.map(function (line, Rating) {
        var arr = line.split("::");
        var r = new Rating(parseInt(arr[0]),
            parseInt(arr[1]),
            parseFloat(arr[2]) - 2.5);
        return r;
    }, [Rating]).cache();

    var model = ALS.train(ratings, 10, 10, 0.01);
    var userRecs = model.recommendProductsForUsers(10);

    var userRecommendedScaled = userRecs.map(function (val, Rating, Tuple) {
        var myRating = Rating; // FIXME: For now make a copy to pass to inner lambda
        var newRatings = val[1].map(function (r) {
            var newRating = Math.max(Math.min(r.rating(), 1.0), 0.0);
            return new Rating(r.user(), r.product(), newRating);
        });
        
        return new Tuple(val[0], newRatings);
    }, [Rating, Tuple]);

    var userRecommended = PairRDD.fromRDD(userRecommendedScaled);

    var binarizedRatings = ratings.map(function (r, Rating) {
        var binaryRating = 0.0;
        if (r.rating() > 0.0) {
            binaryRating = 1.0;
        }

        return new Rating(r.user(), r.product(), binaryRating);
    }, [Rating]);

    var userMovies = binarizedRatings.groupBy(function (r) {
        return r.user();
    });

    var userMoviesList = userMovies.mapValues(function (docs, List) {
        var products = new List();
        docs.forEach(function (r) {
            if (r.rating() > 0.0) {
                products.add(r.product());
            }
        });
        return products;
    }, [List]);

    var userRecommendedList = userRecommended.mapValues(function (docs, List) {
        var products = new List();
        docs.forEach(function (r) {
            products.add(r.product());
        });
        return products;
    }, [List]);

    var relevantDocs = userMoviesList.join(userRecommendedList).values();

    var metrics = RankingMetrics.of(relevantDocs);


 Precision and NDCG at k


In [3]:
    [1, 3, 5].forEach(function (k) {
        print("Precision at " + k + " = " + metrics.precisionAt(k));
        print("NDCG at " + k + " = " + metrics.ndcgAt(k));
    });

    print("Mean average precision = " + metrics.meanAveragePrecision());

    var userProducts = ratings.map(function (r, Tuple) {
        return new Tuple(r.user(), r.product());
    }, [Tuple]);


In [4]:
    var predictions = PairRDD.fromRDD(model.predict(userProducts).map(function (r, Tuple) {
        return new Tuple(new Tuple(r.user(), r.product()), r.rating());
    }, [Tuple]));


In [5]:
    var ratesAndPreds = PairRDD.fromRDD(ratings.map(function (r, Tuple) {
        return new Tuple(new Tuple(r.user(), r.product()), r.rating());
    }, [Tuple])).join(predictions).values();


 Create regression metrics object


In [6]:
    var regressionMetrics = new RegressionMetrics(ratesAndPreds);
    var ret = {};
    ret.RMSE = regressionMetrics.rootMeanSquaredError();
    ret.r2 = regressionMetrics.r2();
var result = ret;


 Root mean squared error


In [7]:
    print("RMSE = " + result.RMSE);


 R-squared


In [8]:
    print("R-squared = " + result.r2);

    sc.stop();
